<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
TEST = False #@param {type: "boolean"}
TEST_INPUT = 'day5_test1.txt'
INPUT = 'day5_input1.txt'



In [ ]:
import re
import time
from dataclasses import dataclass

@dataclass
class SupplyMapping:
  src_start: int
  dst_start: int
  range_length: int

def parse_almanac() -> tuple[list[int], dict[tuple[str, str], list[SupplyMapping]]]:
  input = TEST_INPUT if TEST else INPUT
  with open(input, 'r') as f:
    rows = f.read().splitlines()

  supply_mappings = {}

  seeds_regex = r'^seeds: (( *\d+ *)+)$'
  m = re.match(seeds_regex, rows[0])
  seeds = [int(s) for s in m.group(1).split()]

  category_name_regex = r'^([a-zA-Z]+)-to-([a-zA-Z]+) map:$'
  category_map_regex = r'^(( *\d+ *)+)$'
  row_index = 1
  while row_index < len(rows):
    m = re.match(category_name_regex, rows[row_index])
    row_index +=1
    if not m:
      continue

    src_type = m.group(1)
    dst_type = m.group(2)
    mapping_key = (src_type,  dst_type)
    reverse_mapping_key = mapping_key[::-1]
    supply_mappings.setdefault(mapping_key, [])

    while row_index < len(rows):
      m = re.match(category_map_regex, rows[row_index])
      row_index +=1
      if not m:
        break

      dst_start, src_start, range_length = [int(m) for m in m.group(1).split()]
      supply_mappings[mapping_key].append(
          SupplyMapping(src_start, dst_start, range_length))

  return seeds, supply_mappings

def from_src_to_dst(src_name: str, src_id: int, dst_name: str, supply_mappings: dict[tuple[str,str], list[SupplyMapping]]) -> int:
  seen_supplies = []
  while True:
    mapping_keys = [k for k in supply_mappings.keys() if k[0] == (src_name) and k[1] not in seen_supplies]
    mapping_key = mapping_keys[0]
    temp_dst_name = mapping_key[1]
    seen_supplies.append(src_name)

    for supply_mapping in supply_mappings[mapping_key]:
      if src_id in range(supply_mapping.src_start, supply_mapping.src_start+supply_mapping.range_length):
        dst_id = supply_mapping.dst_start + (src_id - supply_mapping.src_start)
        if temp_dst_name == dst_name:
          return dst_id
        else:
          src_name = temp_dst_name
          src_id = dst_id
          break
    # If we fall off the end of the for loop it's because there was no match
    # for src_id and we should resuse src_id for the next supply.
    src_name = temp_dst_name
    # Return if we've reached the supply we care about.
    if src_name == dst_name:
      return src_id


seeds, supply_mappings = parse_almanac()
print(f'min_location: {min([from_src_to_dst("seed", seed, "location", supply_mappings) for seed in seeds])}')

